In [ ]:
from scipy.spatial import distance as dist
from imutils import face_utils
import imutils
import dlib
import cv2
import time

def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # diagonal eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[4])
    B = dist.euclidean(eye[2], eye[5])

    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])

    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)

    # return the eye aspect ratio
    return ear

# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold
EYE_AR_THRESH = 0.4
EYE_AR_BLINK_FRAMES = 1

# define one constant for the number of consecutive
# frames the eye must be below the threshold for drowsiness
EYE_AR_CONSEC_FRAMES = 20

path_shape_predictor = "./shape_predictor_68_face_landmarks.dat"

# initialize the frame counters for drowsiness
COUNT = 0 

# initialize the frame counters and the total number of blinks
COUNTER = 0
TOTAL = 0

# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(path_shape_predictor)

# create a videoCapture object with a video file or a capture device
cap = cv2.VideoCapture("./sleepyCombination.avi")

# Default resolutions of the frame are obtained.The default resolutions are system dependent.
frame_width = int(cap.get(3))
start_1 = time.time()
start_2 = time.time()
initial_count = 0
count = []
# read until the end of the video frame by frame
while True:
    # cap.read (): decodes and returns the next video frame
    # variable ret: will get the return value by retrieving the camera frame, true or false (via "cap")
    # variable frame: will get the next image of the video (via "cap")
    ret, frame = cap.read()

    if ret:
        # grab the indexes of the facial landmarks for the left eye,
        # right eye respectively
        # The left_eye, right_eye regions from the FACIAL_LANDMARK_IDXS dictionary, 
        # found in the helpers.py script. 
        # These 2-tuple values are stored in left/right eye starting and ending indices.
        (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
        (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
        
        #grayscale the image
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # detect faces in the grayscale frame
        rects = detector(gray, 0)

        # loop over the face detections
        for rect in rects:
            # determine the facial landmarks for the face region, then
            # convert the facial landmark (x, y)-coordinates to a NumPy
            # array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            # extract the left and right eye coordinates, then use the
            # coordinates to compute the eye aspect ratio for both eyes.           
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            # average the eye aspect ratio together for both eyes and mouth
            ear = (leftEAR + rightEAR) / 2.0

            # compute the convex hull for the left and right eye and mouth, then
            # visualize each of the eyes and mouth
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 255), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 255), 1)
    
            #Blink detection
            # check to see if the eye aspect ratio is below the blink
            # threshold, and if so, increment the blink frame counter
            if ear < EYE_AR_THRESH:
                COUNTER += 1
            

            # otherwise, the eye aspect ratio is not below the blink
            # threshold
            else:
                # if the eyes were closed for a sufficient number of
                # then increment the total number of blinks
                if COUNTER >= EYE_AR_BLINK_FRAMES:
                    TOTAL += 1
                    initial_count += 1
                COUNTER = 0
            if time.time() - start_1 > 1:
                start_1 = time.time()
                count.append(initial_count)
                initial_count = 0
                # reset the eye frame counter
                    
            # draw the total number of blinks on the frame along with
            # the computed eye aspect ratio for the frame
            cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)
            cv2.putText(frame, "ECR: {:.2f}".format(ear), (frame_width - 120, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)    
                
            #Drowsiness Detection
            if ear < EYE_AR_THRESH:
                COUNT += 1
                cv2.putText(frame, "Eyes Closed ", (frame_width - 400, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,255), 2)
                # if the eyes were closed for a sufficient number of frames drowsy
                if COUNT >= EYE_AR_CONSEC_FRAMES:
                    cv2.putText(frame, "DROWSINESS ALERT!", (200, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            # otherwise, the eye aspect ratio is not below the blink
            # threshold, so reset the counter and alarm
            else:
                COUNT = 0
                cv2.putText(frame, "Eyes Open ", (frame_width - 400, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (50,255,50), 2)
            if TOTAL-sum(count[:-15])<6 and time.time() - start_2>15:
                
                cv2.putText(frame, "TIRED", (200, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # to display the frame
        cv2.imshow("Output", frame)
        
        # waitKey (n): will wait for keyPress for only n milliseconds and continue to refresh and read the video frame using cap.read ()
        # ord (character): returns an integer representing the Unicode code point of the given Unicode character.
        if cv2.waitKey(1) == ord('q'):
            break
    else:
        break

# to release software and hardware resources
cap.release()

# to close all windows in imshow ()
cv2.destroyAllWindows()